In [174]:
import numpy as np
import random
import math


In [175]:
# la bound
# lb bound
# ht bound


In [176]:
def gen_deadline(exec_time, period):
    # [a,b]
    b = 1.2 * period
    if(exec_time < 10):
        a = exec_time
    elif(exec_time < 100 and exec_time >= 10):
        a = 2*exec_time
    elif(exec_time < 1000 and exec_time >= 100):
        a = 3*exec_time
    else:
        a = 4*exec_time

    return a+random.random()*(b-a)


def gen_tasksets(utilizations, periods):
    """
    Take a list of task utilization sets and a list of task period sets and
    return a list of couples (c, p) sets. The computation times are truncated
    at a precision of 10^-10 to avoid floating point precision errors.
    Args:
        - `utilization`: The list of task utilization sets. For example::
            [[0.3, 0.4, 0.8], [0.1, 0.9, 0.5]]
        - `periods`: The list of task period sets. For examples::
            [[100, 50, 1000], [200, 500, 10]]
    Returns:
        For the above example, it returns::
            [[(30.0, 100), (20.0, 50), (800.0, 1000)],
             [(20.0, 200), (450.0, 500), (5.0, 10)]]
    """
    def trunc(x, p):
        return int(x * 10 ** p) / float(10 ** p)
    [[print(ui) for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)]
    return np.array([[[trunc(ui * pi, 6), trunc(pi, 6), trunc(gen_deadline(trunc(ui * pi, 6), trunc(pi, 6)), 6)] for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)])


In [177]:
def custom_period(n, nsets,max_,min_=0):
	max_power = np.log(max_)


	intervals = np.arange(0,np.floor(max_power)+1)
	intervals = np.append(intervals, max_power)
	#print(intervals)
	task_pp = ((n-1)//(len(intervals)-1))
	
	task_lf = np.mod(n-1, len(intervals)-1).astype(int)
	#print(task_lf,task_pp)
	periods = np.zeros((nsets, n))
	for i in range(nsets):
		for j in range(len(intervals)-1):
			if(j<task_lf):
				periods[i][j*task_pp+j:(j+1)*task_pp+j+1] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp+1)
			else:
				periods[i][j*task_pp+task_lf:(j+1)*task_pp+task_lf] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp)
	periods[0:nsets,-1] =max_ 
	return periods


In [178]:
custom_period(14,2,100)

array([[  1.64804679,   1.30810982,   1.37372558,   6.12438891,
          2.96662742,   7.30901873,   9.32779621,  18.23429394,
         16.96883537,  49.72779376,  33.04281914,  66.19670627,
         93.05911069, 100.        ],
       [  2.49268507,   2.52796187,   1.33612511,   3.64768797,
          3.99364001,   2.96080255,  18.64204259,  19.69945258,
         19.99413364,  28.13048464,  44.76621166,  98.53947291,
         88.36636924, 100.        ]])

In [179]:
def UUniFastDiscard(n, u, nsets):
    sets = []
    while len(sets) < nsets:
        # Classic UUniFast algorithm:
        utilizations = []
        sumU = u
        for i in range(1, n):
            nextSumU = sumU * random.random() ** (1.0 / (n - i))
            utilizations.append(sumU - nextSumU)
            sumU = nextSumU
        utilizations.append(sumU)

        # If no task utilization exceeds 1:
        if all(ut <= 1 for ut in utilizations):
            sets.append(utilizations)

    return sets

    
def gen_uunifastdiscard(nsets, u, n):
    """
    The UUniFast algorithm was proposed by Bini for generating task
    utilizations on uniprocessor architectures.
    The UUniFast-Discard algorithm extends it to multiprocessor by
    discarding task sets containing any utilization that exceeds 1.
    This algorithm is easy and widely used. However, it suffers from very
    long computation times when n is close to u. Stafford's algorithm is
    faster.
    Args:
        - `n`: The number of tasks in a task set.
        - `u`: Total utilization of the task set.
        - `nsets`: Number of sets to generate.
    Returns `nsets` of `n` task utilizations.
    """
    return UUniFastDiscard(n, u, nsets)

In [180]:
def get_taskset(nsets, n):
	u = 0.9
	#n = 1000
	utilizations = gen_uunifastdiscard(nsets, u, n)
	periods = custom_period(n, nsets, 1000, 10)
	return gen_tasksets(utilizations, periods)

	

In [181]:
taskset = get_taskset(1, 8)
taskset

0.06784926886036968
0.020721294268147172
0.03051904732277333
0.12385291994036063
0.36018073048662225
0.06731236082429587
0.21001634644394152
0.01954803185348959


array([[[1.79899000e-01, 2.65146100e+00, 8.46189000e-01],
        [6.74560000e-02, 3.25540400e+00, 3.52156800e+00],
        [2.27362000e-01, 7.44986800e+00, 1.69379000e+00],
        [6.56886700e+00, 5.30376460e+01, 2.23681240e+01],
        [2.65718680e+01, 7.37737080e+01, 7.41915320e+01],
        [2.56143600e+01, 3.80529824e+02, 1.86297973e+02],
        [2.01981333e+02, 9.61741013e+02, 9.94361246e+02],
        [1.95480310e+01, 1.00000000e+03, 6.59841640e+02]]])

In [182]:
taskset_paper = np.array([[6000, 18000, 31000], [2000, 9000, 9800], [1000, 12000, 17000], [
                         90, 3000, 4200], [8, 78, 96], [2, 16, 12], [10, 120, 280], [26, 160, 660]])


In [183]:
def procDemand_func(taskset,t):
    deadlines = taskset[:,1]
    periods = taskset[:,2]
    wcet = taskset[:,0]
    
    #utilizations = wcet/periods #wcet/periods
	#print(deadlines, periods, utilizations)
    #total_utilization = np.sum(utilizations)

    #task_done = (1 + np.floor((t-deadlines)/(periods)))*taskset[:,0]#wcet
    h_t = np.sum(np.maximum(0, 1 + np.floor((t-deadlines)/(periods)))*wcet)

    # task_done[task_done<0] = 0
    # print(task_done)

    # h_t = np.sum(task_done)

    return h_t

In [184]:
t = 16974
ht = procDemand_func(taskset_paper, t)
print(ht)

8890.0


In [185]:
# def get_La(taskset):
#     deadlines = taskset[:,2]
#     periods = taskset[:,1]
#     utilizations = taskset[:,0]/taskset[:,1]
# 	#print(deadlines, periods, utilizations)
#     total_utilization = np.sum(utilizations)
# 	#print(np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
#     La = np.max(np.max(deadlines),np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
#     return La

In [186]:
print(taskset)

[[[1.79899000e-01 2.65146100e+00 8.46189000e-01]
  [6.74560000e-02 3.25540400e+00 3.52156800e+00]
  [2.27362000e-01 7.44986800e+00 1.69379000e+00]
  [6.56886700e+00 5.30376460e+01 2.23681240e+01]
  [2.65718680e+01 7.37737080e+01 7.41915320e+01]
  [2.56143600e+01 3.80529824e+02 1.86297973e+02]
  [2.01981333e+02 9.61741013e+02 9.94361246e+02]
  [1.95480310e+01 1.00000000e+03 6.59841640e+02]]]


In [187]:
taskset_paper = np.array([[6000, 18000, 31000], [2000, 9000, 9800], [1000, 12000, 17000], [90, 3000, 4200], [8, 78, 96], [2, 16, 12], [10, 120, 280], [26, 160, 660]])

In [188]:
def La_bound(taskset):
	"""
	Compute La calculations for the La bound
	
	"""
	wcet = taskset[:,0]
	deadlines = taskset[:,1]
	periods = taskset[:,2]
	utilizations = wcet/periods
	#print("deadlines", deadlines, "periods", periods, "utilizations", utilizations)
	#print(deadlines, periods, utilizations)
	total_utilization = np.sum(utilizations)
	#print("total utilization : " + str(total_utilization))
	#print(np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
	#La = np.maximum(np.max(deadlines),np.max((periods-deadlines))*(total_utilization/(1-total_utilization)))
	print(np.max(deadlines))
	La = np.maximum(np.max(deadlines),np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
	return La


In [189]:
La = La_bound(taskset_paper)
taskset_paper[:,0]/taskset_paper[:,1]
print(La)

18000
18000.0


In [190]:
def Lb_bound(taskset):
    "A synchronous busy period is a processor busy period in which all tasks are release simultaneously at the beginning of the processor busy period, and then, at their maximum rate, and ended by the first processor idle period(the length of such a period can be zero). The length of the synchronous busy period Lb can be computed by the following process"
    wcet = taskset[:,0]
    deadlines = taskset[:,1]
    periods = taskset[:,2]
    w0 = np.sum(wcet)
    a = w0
    b = np.sum(np.ceil(a/periods)*wcet)
    while a != b:
        a = b
        b = np.sum(np.ceil(a/periods)*wcet)
    Lb = b
    
    return Lb

In [191]:
Lb = Lb_bound(taskset_paper)
print(Lb)

16984.0


In [192]:
deadlines = taskset_paper[:, 1]
d_min = np.min(deadlines)
print(d_min)

16


In [205]:
def get_dmin(deadliness, L):
    return np.min(deadliness)
    
    

In [194]:
deadlines = taskset_paper[:, 1]
L = np.minimum(La, Lb)
d_min = get_dmin(deadlines, L)
print(d_min)

# t = np.min(deadlines < L)
# print(t)


12000


In [195]:
taskset_paper.shape

(8, 3)

In [196]:
def get_max_abs_deadline(taskset, L):
    n = taskset.shape[0]
    abs_deadlines = []
    
    for i in range(n):
        p = 0
        j = 0
        Ti = taskset[i][2]
        Di = taskset[i][1]
        while p < L:
            j += 1
            q = p
            p = j*Ti + Di
            
        abs_deadlines.append(q)
    return np.max(abs_deadlines)
    

In [197]:
L = 16984
t = get_max_abs_deadline(taskset_paper, L)

print(t)

16974


In [198]:
def get_max_di(taskset, t):
    n = taskset.shape[0]
    d_max = 0
    
    for j in range(n):
        if taskset[j][1] < t:
            d = np.floor((t - taskset[j][1])/taskset[j][2])*taskset[j][2] + taskset[j][1]
            if d == t:
                d = d - taskset[j][2]
            if d > d_max:
                d_max = d
                
    return d_max
    
    

In [210]:
def qpa(taskset):
    
    La = La_bound(taskset)
    print("La : " + str(La))
    Lb = Lb_bound(taskset)
    print("Lb : " + str(Lb))
    L = min(La, Lb)
    print("L : " + str(L))
    
    t = get_max_abs_deadline(taskset, L)
    print("t : " + str(t))
    h_t = procDemand_func(taskset, t)
    print("h(t) : " + str(h_t))
    d_min = get_dmin(taskset[:,1], L)
    print("d_min : " + str(d_min))
    print("did not enter yet")
    
    while h_t <= t and h_t > d_min:
        h_t = procDemand_func(taskset, t)
        print("came")
        print("t:" + str(t) + "ht" + str(h_t))
        if h_t < t:
            t = h_t
        else:
            d_max = get_max_di(taskset, t)
            t = d_max
    
    print("don't know if it went")
    
    if h_t <= d_min:
        print("Task set is schedulable")
    else:
        print("Task set is not schedulable")
        

    

In [211]:
qpa(taskset_paper)

18000
La : 18000.0
Lb : 16984.0
L : 16984.0
t : 16974
h(t) : 8890.0
d_min : 16
did not enter yet
came
t:16974ht8890.0
came
t:8890.0ht3080.0
came
t:3080.0ht1098.0
came
t:1098.0ht362.0
came
t:362.0ht118.0
came
t:118.0ht26.0
came
t:26.0ht2.0
don't know if it went
Task set is schedulable


In [201]:
taskset_paper = np.array([[6000,18000,31000],[2000,9000,9800],[1000,12000,17000],[90,3000,4200],[8,78,96],[2,16,12],[10,120,280],[26,160,660]])